# 신경망 모델 구성하기 

신경망은 데이터 연산을 수행하는 layer/module로 구성 
torch.nn은 신경망을 구성하는데 필요한 모든 구성요소를 제공 
PyTorch의 모든 모듈은 nn.Module릐 하위 클래스임 
신경망은 다른 모듈로 구성된 모듈

## FashionMNIST 데이터를 이용한 이미지 분류 신경망 구성

In [1]:
import os
import torch
from torch import nn 
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')

Using cpu device


### 클래스 정의하기

nn.Module의 하위클래스 정의 
_ _ init _ _ 에서 신경망 계층 초기화
nn.Module을 상속받은 모든 클래스는 forward 메소드에 입력 데이터에 대한 연산 구현


In [7]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512,512),
        nn.ReLU(),
        nn.Linear(512,10),
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [8]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [11]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f'Predicted class: {y_pred}')

Predicted class: tensor([0])


### 모델 계층(Layer)

In [12]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


#### nn.Flatten

In [13]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


#### nn.Linear

In [14]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


#### nn.ReLU

In [15]:
print(f'Before ReLU: {hidden1}\n\n')
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.4611,  0.2076,  0.0283,  0.3627,  0.1211,  0.3320, -0.5060,  0.0368,
          0.2703,  0.0458, -0.0092, -0.3316, -0.3614,  0.5709,  0.4484, -0.2965,
          0.4407,  0.0771, -0.0154,  0.0765],
        [-0.1391,  0.2437,  0.2972,  0.3329,  0.0672,  0.2467, -0.7685, -0.6131,
         -0.0096,  0.4741, -0.1831, -0.6739, -0.0425,  0.5460,  0.0983, -0.2350,
          0.1905,  0.0655, -0.4400, -0.3596],
        [-0.3925,  0.4616,  0.0277,  0.3376,  0.0447, -0.1159, -0.8630, -0.1513,
         -0.1134,  0.3249, -0.1438, -0.6623,  0.1201,  0.8437,  0.2458, -0.0219,
          0.4586,  0.3298, -0.3228,  0.2676]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.2076, 0.0283, 0.3627, 0.1211, 0.3320, 0.0000, 0.0368, 0.2703,
         0.0458, 0.0000, 0.0000, 0.0000, 0.5709, 0.4484, 0.0000, 0.4407, 0.0771,
         0.0000, 0.0765],
        [0.0000, 0.2437, 0.2972, 0.3329, 0.0672, 0.2467, 0.0000, 0.0000, 0.0000,
         0.4741, 0.0000, 0.0000, 0.0000, 0.5460, 0.09

#### nn.Softmax

In [16]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

### 모델 매개변수 

In [18]:
print("Model structure: ",model, "\n\n")

for name, param in model.named_parameters():
    print(f'Layer: {name} | Size: {param.size()} | Values: {param[:2]}\n ')

Model structure:  NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
) 


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values: tensor([[-0.0034, -0.0174, -0.0138,  ..., -0.0038, -0.0218,  0.0279],
        [ 0.0307, -0.0239, -0.0262,  ...,  0.0312,  0.0252, -0.0091]],
       grad_fn=<SliceBackward0>)
 
Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values: tensor([ 0.0298, -0.0264], grad_fn=<SliceBackward0>)
 
Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values: tensor([[-0.0292,  0.0078, -0.0185,  ...,  0.0283, -0.0081, -0.0246],
        [-0.0199, -0.0137, -0.0053,  ...,  0.0194,  0.0260,  0.0102]],
       grad_fn=<SliceBackward0>)
 
Layer: linear_relu_stack.2.bias | S